In [1]:
# Set your output and project. !!!Must Do before you can proceed!!!
EXPERIMENT_NAME = 'Scikit Learn V1'
PROJECT_NAME =  'erwinh-mldemo'   

In [2]:
import kfp
from kfp import gcp
import os
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

In [3]:
component_url_prefix = 'https://raw.githubusercontent.com/Huize501/kfp/master/scikit-train/' 

In [4]:
dummy_op = kfp.components.load_component_from_url(component_url_prefix + 'component.yaml')# dummy_op = kfp.components.load_component_from_url('http://....../component.yaml')

In [5]:
from kfp import dsl

# Define a pipeline and create a task from a component:
@kfp.dsl.pipeline(name='KFP test erwinh', description='')
def my_pipeline():
    dummy1_task = dummy_op(
        # Input name "Input 1 URI" is converted to pythonic parameter name "input_1_uri"
        pathdata = 'gs://erwinh-mldemo/scikit/marketing-data.csv',
        storage = 'BQ',
        pathoutput ='gs://erwinh-mldemo/scikit/model',
        bqtable = 'erwinh-mldemo.marketing.marketing_propensity_main',
        ).apply(kfp.gcp.use_gcp_secret('user-gcp-sa')) 


In [6]:
pipeline_func = my_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [7]:
#Specify pipeline argument values
arguments = {}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)